In [1]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import shap
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import scipy.stats as stat
import pylab 
import math
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression as LGR,Lasso
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import xgboost as xgb

c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
Fe_CAAC_dataset = df = pd.read_excel("F:\\Nitrogen_activation\\rajaraman\\office\\ofifice\\jupyter\\Extension\\T07\\heatmap\\agostic\\xgboost\\N2_activation_extension_T07_heatmap_agostaic_descrptors.xlsx")

In [17]:
df_1=df.drop(['F00','T01','T02','T03',],axis=1)
df_2=df_1.drop([0,18,9])
df_3=df_1.drop([0,18,9,8,26,17])
df_4=df_1.drop([8,26,17])

In [18]:
X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=20)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)
# transform train and test sets
X_train_scaled_MM = scaler.transform(X_train)
X_test_scaled_MM = scaler.transform(X_test)
X_train_scaled_MM = pd.DataFrame(X_train_scaled_MM, columns=X_train.columns)
X_test_scaled_MM = pd.DataFrame(X_test_scaled_MM, columns=X_test.columns)

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 6],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1],
    'scale_pos_weight': [1, 2]
}

xgb_reg_model= xgb.XGBRegressor()


# Set up Grid Search
grid_search = GridSearchCV(
    estimator=xgb_reg_model,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)


# Fit the model
grid_search.fit(X_train_scaled_MM, y_train)


#print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)



Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Best score:  -252.08297865129967
Best parameters found:  {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'scale_pos_weight': 1, 'subsample': 1.0}


In [14]:
X

,F03,F08
1,2.16440,-0.26690
2,2.11500,-0.27940
3,2.11830,-0.27220
4,1.98800,-0.45250
5,1.95700,-0.45890
6,1.90100,-0.41270
7,1.87235,-0.31450
10,2.16400,-0.26890
11,2.11100,-0.27420
12,2.09600,-0.27320


In [19]:
xgb_reg_model = xgb.XGBRegressor(n_estimators=best_params['n_estimators'],learning_rate=best_params['learning_rate'],max_depth=best_params['max_depth'],min_child_weight=best_params['min_child_weight'],subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],gamma=best_params['gamma'],scale_pos_weight=best_params['scale_pos_weight'])

xgb_reg_model.fit(X_train_scaled_MM,y_train)
# prediction on Training data
training_data_prediction_X_train_xgb_reg = xgb_reg_model.predict(X_train_scaled_MM)
# prediction on Test data
test_data_prediction_X_test_xgb_reg = xgb_reg_model.predict(X_test_scaled_MM)
error_score_r2 = metrics.r2_score(y_train, training_data_prediction_X_train_xgb_reg)
error_score_MAE = metrics.mean_absolute_error(y_train, training_data_prediction_X_train_xgb_reg)
error_score_MSE = metrics.mean_squared_error(y_train, training_data_prediction_X_train_xgb_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_train,training_data_prediction_X_train_xgb_reg)))
# R squared Error
error_score_r2 = metrics.r2_score(y_test, test_data_prediction_X_test_xgb_reg)
error_score_MAE = metrics.mean_absolute_error(y_test, test_data_prediction_X_test_xgb_reg)
error_score_MSE = metrics.mean_squared_error(y_test, test_data_prediction_X_test_xgb_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_test,test_data_prediction_X_test_xgb_reg)))

R squared Error :  0.9748566582934849
mean_absolute_error :  4.599492720627323
mean_squared_error :  43.96611454766233
RMSE 6.630694876682408
R squared Error :  0.8626998854663845
mean_absolute_error :  9.01933939224332
mean_squared_error :  97.20894876189467
RMSE 9.859459861569226


In [11]:
# can tune other metrics, such as MSE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X)
# transform train and test sets
X_scaled_MM = scaler.transform(X)
X_scaled_MM = pd.DataFrame(X_scaled_MM, columns=X.columns)

scores = cross_val_score(xgb_reg_model, X_scaled_MM, y, cv=2)
print(scores)
print(np.mean(scores))

scores = cross_val_score(xgb_reg_model, X_scaled_MM, y, cv=3)
print(scores)
print(np.mean(scores))

scores = cross_val_score(xgb_reg_model, X_scaled_MM, y, cv=4)
print(scores)
print(np.mean(scores))

scores = cross_val_score(xgb_reg_model, X_scaled_MM, y, cv=5)
print(scores)
print(np.mean(scores))


[0.78440754 0.86508419]
0.8247458677438804
[0.89587886 0.90281673 0.73868352]
0.8457930367073585
[0.80617948 0.84501395 0.94260028 0.66478819]
0.8146454766750574
[0.68795268 0.9445588  0.80438588 0.91557745 0.49085154]
0.768665271262571


In [10]:

# Initialize the Leave-One-Out cross-validator
loo = LeaveOneOut()

X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model


# Apply MinMax Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)  # Scale the features and store them in X_scaled


# Initialize lists to store predictions and actual values
y_pred_loo = []
y_true_loo = []

# Loop over each training/test split
for train_index, test_index in loo.split(X_scaled):
    X_train_loo, X_test_loo = X_scaled[train_index], X_scaled[test_index]
    y_train_loo, y_test_loo = y.iloc[train_index], y.iloc[test_index]
    
    # Train the Lasso model
xgb_reg_model = xgb.XGBRegressor(n_estimators=best_params['n_estimators'],learning_rate=best_params['learning_rate'],max_depth=best_params['max_depth'],min_child_weight=best_params['min_child_weight'],subsample=best_params['subsample'],colsample_bytree=best_params['colsample_bytree'],gamma=best_params['gamma'],scale_pos_weight=best_params['scale_pos_weight'])
xgb_reg_model.fit(X_train_loo, y_train_loo)
    
# Predict the left-out sample
y_pred_loo.append(xgb_reg_model.predict(X_test_loo)[0])
y_true_loo.append(y_test_loo.values[0])

# Convert lists to arrays
y_pred_loo = np.array(y_pred_loo)
y_true_loo = np.array(y_true_loo)

# Calculate LOO performance metrics
Q2_loo = r2_score(y_true_loo, y_pred_loo)
MAE_loo = mean_absolute_error(y_true_loo, y_pred_loo)
MSE_loo = mean_squared_error(y_true_loo, y_pred_loo)
RMSE_loo = np.sqrt(MSE_loo)

# Print the results
print("LOO Q^2 score: ", Q2_loo)
print("LOO Mean Absolute Error: ", MAE_loo)
print("LOO Mean Squared Error: ", MSE_loo)
print("LOO Root Mean Squared Error: ", RMSE_loo)



LOO Q^2 score:  nan
LOO Mean Absolute Error:  8.615331390151596
LOO Mean Squared Error:  74.22393496213142
LOO Root Mean Squared Error:  8.615331390151596


c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
